In [56]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
X = newdf.to_numpy()
y = df['CO2E'].to_numpy()

In [223]:
"""searchSpace = { -------------------------This one for xgboost
    "n_estimators": [75],
    "max_depth": [8, 9, 10],
    "eta": [.1],
    "min_child_weight": [0],
    "gamma": [0.15, 0.2, 0.25],
    "seed": [42]
}"""
searchSpace = {
    "max_depth": [9, 10, 11],
    "min_samples_split": [2],
    "min_samples_leaf": [1],
    "n_estimators": [50, 75, 100, 150],
    "max_features": ['sqrt', 'log2', None],
    "bootstrap": [True, False]

}

scoring = {
    'r2': 'r2',  
    'neg_mean_squared_error': 'neg_mean_squared_error'
}

model = RandomForestRegressor()


In [240]:
GS = GridSearchCV(estimator=model,
                  param_grid= searchSpace,
                  scoring = scoring,
                  refit="r2",
                  cv=5,
                  verbose=4)

In [241]:
GS.fit(X, y)


Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV 1/5] END bootstrap=True, max_depth=9, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; neg_mean_squared_error: (test=-198.393) r2: (test=-1.000) total time=   0.0s
[CV 2/5] END bootstrap=True, max_depth=9, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; neg_mean_squared_error: (test=-164.442) r2: (test=0.226) total time=   0.0s
[CV 3/5] END bootstrap=True, max_depth=9, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; neg_mean_squared_error: (test=-173.462) r2: (test=0.412) total time=   0.0s
[CV 4/5] END bootstrap=True, max_depth=9, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; neg_mean_squared_error: (test=-37.385) r2: (test=0.616) total time=   0.0s
[CV 5/5] END bootstrap=True, max_depth=9, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=50; neg_mean_squared_error: (test=-30.842) 

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'bootstrap': [True, False], 'max_depth': [9, 10, 11],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1], 'min_samples_split': [2],
                         'n_estimators': [50, 75, 100, 150]},
             refit='r2',
             scoring={'neg_mean_squared_error': 'neg_mean_squared_error',
                      'r2': 'r2'},
             verbose=4)

In [242]:
GS.best_params_

{'bootstrap': False,
 'max_depth': 9,
 'max_features': 'log2',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [243]:
GS.best_score_

0.49481282243233854

In [244]:
GS.best_estimator_

RandomForestRegressor(bootstrap=False, max_depth=9, max_features='log2')

In [245]:
ndf = pd.DataFrame(GS.cv_results_)
ndf.to_csv("cv_results.csv")